## <font color='blue'>SME0104 - Cálculo Numérico</font>

### Primeiro Trabalho em Grupo 
#### Comparação de Métodos na Solução do Laplaciano em Grafos para propagação de informação

**Luis Gustavo Nonato**

------

Considere os arquivos `manh.el` e `manh.xy` que fornecem as arestas e as coordenadas dos vértices do grafo de ruas da ilha de Manhattan, NY (arquivos disponíveis para download no Google Drive).

O grafo de ruas possui diversas componentes conexas, considerando somente a maior componente conexa, você deve realizar as seguintes tarefas:
1. Selecione alguns vértices do grafo $v_{i_1},v_{i_2},\ldots,v_{i_k},\, k<<n$ ($n$ é o número de vértices na maior componente do grafo e $k$ é um número bem menor que $n$, $k=10$ por exemplo) e atribua valores distindos $c_{i_1},c_{i_2},\ldots,c_{i_k}$ a cada um dos vértices selecionados (por exemplo valores no intervalo (0,10]);
2. Construa a matriz Laplaciana $L$ do grafo de ruas;
3. Construa a matriz de penalidades $P$, sendo $P$ é uma matriz diagonal onde a entrada $P_{jj}=\alpha$ se $j$ corresponde ao índice de algum dos vértices escolhidos no item 1 acima ($\alpha=1.0e7$ por exemplo), sendo $P_{ii}=0$ caso contrário.
4. Construa um vetor $b$ da seguinte forma:
$$
b_{j} = \left\{\begin{matrix} c_{i_s} & \text{ se } j = i_s \\ 0 & \text{ caso contrário}\end{matrix}\right.
$$
5. Compare o tempo de solução do sistema
$$
(L+P)x = Px
$$
para os métodos:
    - Decomposição LU
    - Cholesky
    - Jacobi e Gaus-Seidel
    - Gradientes Conjugados
    
6. Refaça as tarefas com representação por matriz esparsa e matrizes cheias, comparando os resultados.

In [5]:
# Solucao


import matplotlib.pyplot as plt
  
names = []
marks = []
  
f = open('./dados/manh.el','r')
for row in f:
    row = row.split('\t')
    print(row)
  
# print(names)
# print(marks)
# plt.bar(names, marks, color = 'g', label = 'File Data')
  
# plt.xlabel('Student Names', fontsize = 12)
# plt.ylabel('Marks', fontsize = 12)
  
# plt.title('Students Marks', fontsize = 20)
# plt.legend()
# plt.show()

['0', '1803\n']
['0', '69\n']
['0', '2714\n']
['1', '7977\n']
['1', '3695\n']
['2', '6481\n']
['2', '5577\n']
['3', '5948\n']
['3', '1095\n']
['4', '3531\n']
['5', '2570\n']
['5', '1172\n']
['5', '2830\n']
['6', '8341\n']
['6', '8156\n']
['7', '7337\n']
['7', '5686\n']
['8', '5280\n']
['9', '8066\n']
['9', '5309\n']
['10', '8016\n']
['10', '3994\n']
['10', '4852\n']
['10', '1155\n']
['11', '7358\n']
['11', '8206\n']
['11', '339\n']
['11', '8781\n']
['12', '7906\n']
['12', '2238\n']
['12', '7249\n']
['13', '7093\n']
['13', '5804\n']
['14', '372\n']
['14', '3339\n']
['14', '6274\n']
['15', '7529\n']
['15', '1405\n']
['15', '6299\n']
['15', '2799\n']
['16', '6074\n']
['16', '4135\n']
['17', '6252\n']
['17', '8041\n']
['18', '2936\n']
['18', '7869\n']
['19', '8368\n']
['19', '4809\n']
['19', '3872\n']
['19', '3375\n']
['20', '908\n']
['20', '2788\n']
['20', '187\n']
['20', '1390\n']
['21', '6479\n']
['21', '4527\n']
['21', '7556\n']
['21', '5076\n']
['22', '87\n']
['22', '7301\n']
['22', '